In [1]:
import Pkg 
Pkg.add("BenchmarkTools")
using BenchmarkTools
Pkg.add("PyCall")
using PyCall
Pkg.add("Conda")
using Conda
Conda.add("numpy")
using Libdl

    Updating registry at `~/.julia/registries/General`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.6/Project.toml`
  No Changes to `~/.julia/environments/v1.6/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.6/Project.toml`
  No Changes to `~/.julia/environments/v1.6/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.6/Project.toml`
  No Changes to `~/.julia/environments/v1.6/Manifest.toml`
┌ Info: Running `conda install -y numpy` in root environment
└ @ Conda /home/hpereira/.julia/packages/Conda/sNGum/src/Conda.jl:128


Solving environment: ...working... done

# All requested packages already installed.



In [2]:
a = rand(10^7) ;# Vetor unidimensional com número aleatórios.
sum(a)

d = Dict()

Dict{Any, Any}()

## C language

In [3]:

C_code = """
#include <stddef.h>
double c_sum(size_t n, double *X) {
    double s = 0.0;
    for (size_t i = 0; i < n; ++i) {
        s += X[i];
    }
    return s;
}
"""

const Clib = tempname()  

open(`gcc -fPIC -O3 -msse3 -xc -shared -o $(Clib * "." * Libdl.dlext) -`, "w") do f
    print(f, C_code) 
end

# definindo a função em Julia que chama a função em C :
c_sum(X::Array{Float64}) = ccall(("c_sum", Clib), Float64, (Csize_t, Ptr{Float64}), length(X), X)

c_bench = @benchmark c_sum($a)

d["C"] = median(c_bench.times) / 1e6  # in milliseconds

14.2300035

## C com a biblioteca -ffast-math 

In [4]:
const Clib_fastmath = tempname()   # make a temporary file

open(`gcc -fPIC -O3 -msse3 -xc -shared -ffast-math -o $(Clib_fastmath * "." * Libdl.dlext) -`, "w") do f
    print(f, C_code) 
end

c_sum_fastmath(X::Array{Float64}) = ccall(("c_sum", Clib_fastmath), Float64, (Csize_t, Ptr{Float64}), length(X), X)

c_fastmath_bench = @benchmark $c_sum_fastmath($a)

d["C ffast-math"] = median(c_fastmath_bench.times)/1e6

6.600298

## Python com função pura

In [5]:
pysum = pybuiltin("sum")

py_list_bench = @benchmark $pysum($a)

d["Python- Built-in"] = median(py_list_bench.times)/1e6

1857.948191

## Python com **Numpy**

In [6]:


numpy_sum = pyimport("numpy")["sum"]

py_numpy_bench = @benchmark $numpy_sum($a)

d["Python numpy"] = median(py_numpy_bench.times) / 1e6

4.4598555

## Python definindo a função

In [7]:
py"""
def py_sum(A):
    s = 0.0
    for a in A:
        s += a
    return s
"""

sum_py = py"py_sum"

py_hand = @benchmark $sum_py($a)

d["Python hand-written"] = median(py_hand.times) / 1e6


2128.739009

## Julia Built-in

In [8]:
j_bench = @benchmark sum($a)
d["Julia built-in"] = median(j_bench.times) / 1e6


4.438667

## Julia definindo a função

In [9]:
function mysum(A)   
    s = 0.0 # s = zero(eltype(a))
    for a in A
        s += a
    end
    s
end

j_bench_hand = @benchmark mysum($a)

d["Julia hand-written"] = median(j_bench_hand.times) / 1e6

11.168012

## Resultados

In [10]:
for (key, value) in sort(collect(d), by=last)
    println(rpad(key, 30, "."), lpad(round(value; digits=1), 6, "."), "ms")
end

Julia built-in...................4.4ms
Python numpy.....................4.5ms
C ffast-math.....................6.6ms
Julia hand-written..............11.2ms
C...............................14.2ms
Python- Built-in..............1857.9ms
Python hand-written...........2128.7ms
